In [9]:
from dateutil.parser import parse
from typing import Any
from datetime import datetime
from pydantic import BaseModel, field_validator
import pytz

In [13]:
class Model(BaseModel):
    dt: datetime
    
    @field_validator("dt", mode='before')
    @classmethod
    def parse_datetime(cls, value: Any):
        if isinstance(value, str):
            print("parsing string")
            try: 
                return parse(value)
            except Exception as ex:
                raise ValueError(str(ex))
        print("pass through...")
        return value

In [14]:
Model(dt=100_000)

pass through...


Model(dt=datetime.datetime(1970, 1, 2, 3, 46, 40, tzinfo=TzInfo(UTC)))

In [15]:
Model(dt="2020/1/1 3pm")

parsing string


Model(dt=datetime.datetime(2020, 1, 1, 15, 0))

In [16]:
class Model(BaseModel):
    dt: datetime
    
    @field_validator("dt", mode='before')
    @classmethod
    def parse_datetime(cls, value: Any) -> Any:
        if isinstance(value, str):
            print("parsing string")
            try: 
                return parse(value)
            except Exception as ex:
                raise ValueError(str(ex))
        print("pass through...")
        return value
    
    @field_validator("dt")
    @classmethod
    def make_utc(cls, dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        return dt

In [20]:
Model(dt=100_000)

pass through...


Model(dt=datetime.datetime(1970, 1, 2, 3, 46, 40, tzinfo=<UTC>))

In [21]:
Model(dt="2020/1/1 3pm")

parsing string


Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

In [24]:
class Model(BaseModel):
    number: int
    
    @field_validator("number")
    @classmethod
    def add_1(cls, value: int):
        print(f"Running add_1({value}) -> {value + 1}")
        return value + 1
    
    @field_validator("number")
    @classmethod
    def add_2(cls, value: int):
        print(f"Running add_2({value}) -> {value + 2}")
        return value + 2
    
    @field_validator("number", mode='before')
    def subtract_1(cls, value):
        print(f"Running subtract_1({value}) -> {value - 1}")
        return value - 1
    
    @field_validator("number", mode='before')
    def subtract_2(cls, value):
        print(f"Running subtract_2({value}) -> {value - 2}")
        return value - 2

In [26]:
Model(number=0)

Running subtract_2(0) -> -2
Running subtract_1(-2) -> -3
Running add_1(-3) -> -2
Running add_2(-2) -> 0


Model(number=0)